## 3. 无重复字符的最长子串

滑窗 + 哈希表

In [37]:
from collections import defaultdict
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        cnt = defaultdict(int)
        left = right = 0
        ans = 0
        while right < len(s):
            # print(cnt)
            cur = s[right]
            cnt[cur] += 1
            while cnt[cur] > 1 and left <= right:
                cnt[s[left]] -= 1
                left += 1
            right += 1
            ans = max(ans, right - left)
        return ans

In [38]:
s = ""

In [39]:
S = Solution()
S.lengthOfLongestSubstring(s)

0

也可以用set代替哈希表的作用

## 206. 反转链表

链表常见双指针

In [40]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [43]:
from typing import Optional
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        pre, cur = None, head
        while cur:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        return pre

### 递归

In [45]:
class Solution:
    def reverseList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        def reverse(pre, cur):
            if not cur: return pre
            nxt = cur.next
            cur.next = pre
            return reverse(cur, nxt)
        return reverse(None, head)

## 146. LRU缓存 (重要)

双向链表 + 哈希表

哈希表中存放双向链表的节点，这样就能以 O(1) 的复杂度进行查询；定位到要操作的节点之后，双链表的增删改都是 O(1) 的复杂度。 同时双向链表中存储键值对，查询映射关系。

In [9]:
class DLinkedNode:
    """双向链表"""
    def __init__(self, key = 0, value = 0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.head = DLinkedNode()
        self.tail = DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0
        self.cache = dict()
        
    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        node = self.cache[key]
        self.moveToHead(node)
        return node.value
        
    def put(self, key: int, value: int) -> None:
        if key not in self.cache:
            node = DLinkedNode(key, value)
            self.cache[key] = node
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                removed = self.removeTail()
                self.cache.pop(removed.key)
                self.size -= 1
        else:
            node = self.cache[key]
            node.value = value
            self.moveToHead(node)
    
    def addToHead(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node
        
    def removeNode(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
    
    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)
        
    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node
        
        

## 215. 数组中的第K个最大元素

### 快速选择

In [13]:
import random
from typing import List
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        def partition(nums, l, r):
            cur = random.randint(l, r)
            x, i = nums[cur], l
            nums[cur], nums[r] = nums[r], nums[cur]
            for j in range(l, r):
                if nums[j] <= x:
                    nums[j], nums[i] = nums[i], nums[j]
                    i += 1
            nums[r], nums[i] = nums[i], nums[r]
            return i
        
        def quick_select(nums, l, r, k):
            idx = partition(nums, l, r)
            if idx == k: return nums[idx]
            return quick_select(nums, idx + 1, r, k) if idx < k else\
            quick_select(nums, l, idx - 1, k)
        
        return quick_select(nums, 0, len(nums) - 1, len(nums) - k)

### 最小堆

In [16]:
import heapq
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        heap = []
        for n in nums:
            if len(heap) == k: heapq.heappushpop(heap, n)
            else: heapq.heappush(heap, n)
        return heap[0]

## 25. K个一组反转链表

重点在于细节的考虑

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [3]:
class Solution:
    def reverse(self, head: ListNode, tail: ListNode):
        """翻转一个子链表，并且返回新的头与尾"""
        pre = tail.next
        cur = head
        while pre != tail:
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt
        return tail, head
    
    def reverseKGroup(self, head: ListNode, k: int) -> ListNode:
        dumb = ListNode(0, head)
        pre = dumb
    
        while head:
            tail = pre
            # 判断剩余部分长度是否大于等于 k
            # 大于的话定位到 要翻转的最后一个节点，否则之间返回链表的头节点
            for _ in range(k):
                tail = tail.next
                if not tail:
                    return dumb.next
            nxt = tail.next
            head, tail = self.reverse(head, tail)
            # 把子链表重新接回原链表
            pre.next = head
            tail.next = nxt
            pre = tail
            head = tail.next
            
        return dumb.next

## 15. 三数之和

要点：在第二层遍历中使用双指针反向查询目标和

In [8]:
from typing import List
class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        nums.sort()
        n, res = len(nums), []
        for i in range(n):
            if i > 0 and nums[i] == nums[i - 1]: continue
            tgt = -nums[i]
            k = n - 1  # 每次都重新枚举一遍 k
            for j in range(i + 1, n):
                if j > i + 1 and nums[j] == nums[j - 1]: continue
                while j < k + 1 and nums[j] + nums[k] > tgt: k -= 1
                if j == k: break  # 指针重合时特判，直接 break
                if nums[j] + nums[k] == tgt:
                    res.append([nums[i], nums[j], nums[k]])
        return res

## 53. 最大子数组和

### 动态规划

思路：考虑以 nums[i] 为结尾的最大子数组和，则 nums[i] 可以单独成段或与前面的数字一起，即：

dp[i] = max(nums[i], dp[i - 1] + nums[i])， dp[-1] = -∞，遍历时维护最大值即可。

In [9]:
class Solution:
    def maxSubArray(self, nums: List[int]) -> int:
        ans = nums[0]
        pre = -float("Inf")
        for n in nums:
            pre = n if pre < 0 else pre + n
            ans = max(ans, pre)
        return ans

## 912. 排序数组

In [48]:
from random import randint
from typing import List
class Solution:
    def sortArray(self, nums: List[int]) -> List[int]:
        def quick_sort(l, r):
            if l < r:
                x, cur = randint(l, r), l
                nums[x], nums[r] = nums[r], nums[x]
                for i in range(l, r):
                    if nums[i] < nums[r]:
                        nums[cur], nums[i] = nums[i], nums[cur]
                        cur += 1
                nums[cur], nums[r] = nums[r], nums[cur]
                quick_sort(l, cur - 1)
                quick_sort(cur + 1, r)
        quick_sort(0, len(nums) - 1)
        return nums

In [27]:
nums = [5,1,1,2,0,0]

In [28]:
s = Solution()
s.sortArray(nums)

[0, 0, 1, 1, 2, 5]

原始的快排过不了最后一个重复数字很多的例子，需用到三路快排

### 三路快排

将排序的数据分为三部分，小于 x 等于 x 和大于 x，这样只需对前后两部分进行排序即可，用于解决重复数字较多的情况。

In [26]:
from random import randint
from typing import List
class Solution:
    def sortArray(self, nums: List[int]) -> List[int]:
        def quick_sort(arr, l, r):
            if l >= r:
                return
            random_index = random.randint(l, r)
            pivot = arr[random_index]
            arr[l], arr[random_index] = arr[random_index], arr[l]
            i = l + 1
            j = l 
            k = r + 1
            while i < k:
                if arr[i] < pivot:
                    arr[i], arr[j + 1] = arr[j + 1], arr[i]
                    j += 1
                    i += 1
                elif arr[i] > pivot:
                    arr[i], arr[k - 1] = arr[k - 1], arr[i]
                    k -= 1
                else: 
                    i += 1
            arr[l], arr[j] = arr[j], arr[l]
            quick_sort(arr, l, j - 1)
            quick_sort(arr, k, r)
                
        quick_sort(nums, 0, len(nums) - 1)
        return nums

In [7]:
import random
def quick_sort(arr, l, r):
    if l >= r:
        return
    random_index = random.randint(l, r)
    pivot = arr[random_index]
    arr[l], arr[random_index] = arr[random_index], arr[l]
    i = l + 1
    j = l 
    k = r + 1
    while i < k:
        if arr[i] < pivot:
            arr[i], arr[j + 1] = arr[j + 1], arr[i]
            j += 1
            i += 1
        elif arr[i] > pivot:
            arr[i], arr[k - 1] = arr[k - 1], arr[i]
            k -= 1
        else: 
            i += 1
    arr[l], arr[j] = arr[j], arr[l]
    quick_sort(arr, l, j - 1)
    quick_sort(arr, k, r)

In [19]:
arr = [5,4,3,2,1,0]
quick_sort(arr, 0, 5)
arr

[0, 1, 2, 3, 4, 5]

## 21. 合并两个有序链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

### 迭代

In [3]:
from typing import Optional
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        dumb = ListNode()
        cur = dumb
        while list1 and list2:
            if list1.val <= list2.val:
                cur.next = list1
                list1 = list1.next
            else:
                cur.next = list2
                list2 = list2.next
            cur = cur.next
        if list1: cur.next = list1
        if list2: cur.next = list2
        return dumb.next

### 递归

In [6]:
from typing import Optional
class Solution:
    def mergeTwoLists(self, list1: Optional[ListNode], list2: Optional[ListNode]) -> Optional[ListNode]:
        if not list1: return list2
        if not list2: return list1
        if list1.val <= list2.val:
            list1.next = self.mergeTwoLists(list1.next, list2)
        else: list2.next = self.mergeTwoLists(list1, list2.next)
        return list1 if list1.val <= list2.val else list2

## 20. 有效的括号

In [10]:
class Solution:
    def isValid(self, s: str) -> bool:
        tab = {")": "(", "]": "[", "}": "{"}
        stk = []
        for c in s:
            if c in tab:
                if not stk or tab[c] != stk.pop(): 
                    return False
            else: stk.append(c)
        return not stk

## 121. 买卖股票的最佳时机

In [12]:
from typing import List
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        min_ = float("inf")
        ans = 0
        for n in prices:
            min_ = min(min_, n)
            ans = max(ans, n - min_)
        return ans

## 141. 环形链表

In [13]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [16]:
class Solution:
    def hasCycle(self, head: Optional[ListNode]) -> bool:
        slow = fast = head
        while fast and fast.next and fast.next.next:
            slow = slow.next
            fast = fast.next.next
            if slow == fast:
                return True
        return False

## 5. 最长回文串

In [18]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        def expand(l, r):
            while l >= 0 and r < n and s[l] == s[r]:
                l -= 1
                r += 1
            return l + 1, r  # 返回的值刚好可以作为索引
        
        n = len(s)
        left = right = 0
        for i in range(n):
            l1, r1 = expand(i, i + 1)
            l2, r2 = expand(i, i)
            if r1 - l1 > right - left: left, right = l1, r1
            if r2 - l2 > right - left: left, right = l2, r2
            
        return s[left: right]

## [236. 二叉树的最近公共祖先](https://leetcode.cn/problems/lowest-common-ancestor-of-a-binary-tree/description/)

In [19]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

In [29]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(node, path):
            if node == p or node == q:
                res.append(path)
            if node.left:
                dfs(node.left, path + [node.left])
            if node.right:
                dfs(node.right, path + [node.right])
        res = []
        dfs(root, [root])
        res.sort(key = lambda x: len(x))
        # print(res)
        for node in res[1]:
            if node in res[0]:
                ans = node
        return ans

### 存储父节点

记录所有节点的父节点，从 `p` 开始不断往上跳，记录经过的结点，再从 `q` 开始往上跳，发现记录过的节点即为最近公共祖先。

In [1]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(node, fa):
            cnt[node] = fa
            if node.left:
                dfs(node.left, node)
            if node.right:
                dfs(node.right, node)
        
        cnt = {}
        dfs(root, None)
        vis = set()
        cur = p
        while cur:
            vis.add(cur)
            cur = cnt[cur]
        cur = q
        while cur:
            if cur in vis: return cur
            cur = cnt[cur]
        return 

### 巧妙的递归

一个节点 node 是 LCA 当且仅当：

1. node 是 p 或 q，且另一个节点在它的左子树或右子树中。
2. p 和 q 分别在 node 的左子树与右子树中。

一个节点是 “有用的（`return True`）” 当且仅当：

1. node 是 p 或 q，或 p/q 在它的 左/右 子树中。

> 我们递归遍历整棵二叉树，定义 $f_x$ 表示 $x$ 节点的子树中是否包含 $p$ 节点或 $q$ 节点，如果包含为 $true$，否则为 $false$。那么符合条件的最近公共祖先 $x$ 一定满足如下条件：
$$(f_{lson}\ \&\&\ f_{rson})\ ||\ (x = p || x = q)\ \&\&\ (f_{lson}\ ||\ f_{rson})$$

In [40]:
class Solution:
    def lowestCommonAncestor(self, root: 'TreeNode', p: 'TreeNode', q: 'TreeNode') -> 'TreeNode':
        def dfs(x, p, q):
            nonlocal ans
            if not x: return False
            lson = dfs(x.left, p, q)
            rson = dfs(x.right, p, q)
            # 满足LCA节点的条件
            if (lson and rson) or (x.val == p.val or x.val == q.val) and (lson or rson):
                ans = x
            # 每个节点应当返回的值
            return lson or rson or x.val == p.val or x.val == q.val
        ans = None
        dfs(root, p, q)
        return ans

## 54. 螺旋矩阵

nb 做法

In [6]:
from typing import List
class Solution:
    def spiralOrder(self, matrix: List[List[int]]) -> List[int]:
        res = []
        while matrix:
            # 削头
            res += matrix.pop(0)
            # 顺时针旋转 90 度再削头
            # list(zip(*matrix)) 的作用为将矩阵逆时针旋转 90 度
            # [::-1]再翻转，相当于顺时针 90 度
            matrix = list(zip(*matrix))[::-1]
        return res

## 23. 合并 k 个升序链表 

In [16]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [19]:
from collections import deque
from typing import Optional

class Solution:
    
    def merge(self, l1, l2):
        if not l1: return l2
        if not l2: return l1
        if l1.val < l2.val: l1.next = self.merge(l1.next, l2)
        else: l2.next = self.merge(l1, l2.next)
        return l1 if l1.val < l2.val else l2
        
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        dumb = ListNode()
        lists = deque(lists)
        while lists:
            cur1 = lists.pop()
            if not lists: return cur1
            cur2 = lists.pop()
            lists.append(self.merge(cur1, cur2))
        return

## 92. 反转链表Ⅱ

In [20]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [21]:
class Solution:
    def reverseBetween(self, head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
        def reverse(l, r):
            if l != r:
                pre, cur = l, l.next
                while pre != r:
                    cur.next = pre
                    pre = cur
                    cur = cur.next
            return r, l
        
        cnt = 0
        dumb = ListNode()
        cur = dumb
        while cur:
            cnt += 1
            if cnt == left: l = cur
            if cnt == right + 1: r = cur
            cur = cur.next
        nr = r.next
        ll, rr = reverse(l.next, r)
        l.next = ll; rr.next = nr
        
        return dumb.next

### 只遍历一次

In [23]:
class Solution:
    def reverseBetween(self, head: ListNode, left: int, right: int) -> ListNode:
        # 设置 dummyNode 是这一类问题的一般做法
        dummy_node = ListNode(-1)
        dummy_node.next = head
        pre = dummy_node
        for _ in range(left - 1):
            pre = pre.next

        cur = pre.next
        for _ in range(right - left):
            nxt = cur.next
            cur.next = nxt.next
            nxt.next = pre.next
            pre.next = nxt
        return dummy_node.next

看不懂

## 415. 字符串相加

In [46]:
class Solution:
    def addStrings(self, num1: str, num2: str) -> str:
        res = []; c = 0
        p1, p2 = len(num1), len(num2)
        while p1 > 0 or p2 > 0:
            n1 = n2 = 0
            if p1 > 0: n1 = int(num1[p1 - 1])
            if p2 > 0: n2 = int(num2[p2 - 1])
            nc, cur = divmod(n1 + n2 + c, 10)
            res.append(str(cur))
            c = nc
            p1 -= 1; p2 -= 1
        if c: res.append("1")
        return "".join(res[::-1])

In [49]:
num1 = "1"; num2 = "999999999999999"

In [50]:
s = Solution()
s.addStrings(num1, num2)

'1000000000000000'

## 142. 环形链表

In [51]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [53]:
class Solution:
    def detectCycle(self, head: Optional[ListNode]) -> Optional[ListNode]:
        vis = set(); cur = head
        while cur:
            if cur in vis: return cur
            vis.add(cur)
            cur = cur.next
        return

### 可以空间复杂度为O(1)，但是看不懂

## 143. 重排链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [59]:
from collections import deque
from typing import Optional
class Solution:
    def reorderList(self, head: Optional[ListNode]) -> None:
        """
        Do not return anything, modify head in-place instead.
        """
        res = deque()
        p = head.next
        while p:
            res.append(p)
            p = p.next
        cur = head
        for i in range(len(res)):
            if i & 1: cur.next = res.popleft()
            else: cur.next = res.pop()
            cur = cur.next    
        cur.next = None

In [60]:
a = [1,2,3,4]
head = modules.constructList(a)

In [61]:
s = Solution()
s.reorderList(head)

In [62]:
while head:
    print(head.val)
    head = head.next

1
4
2
3


## 124. 二叉树中的最大路径和

In [63]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [66]:
class Solution:
    def maxPathSum(self, root: Optional[TreeNode]) -> int:
        def dfs(node):
            nonlocal ans
            if not node: return 0
            cur = node.val
            left = dfs(node.left)
            right = dfs(node.right)
            # 维护最大路径和的信息
            ans = max(ans, cur + max(left, 0) + max(right, 0))
            # 对于父节点的贡献只能返回两条路径中较大的那一条，否则不符合路径的定义
            return cur + max(max(left, right), 0)
        
        ans = float("-inf")
        dfs(root)
        return ans

## 二叉树的中序遍历

In [68]:
from typing import List
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        def dfs(node):
            if not node: return
            dfs(node.left)
            res.append(node.val)
            dfs(node.right)
        return res

In [69]:
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        stk = []
        cur = root
        while stk or cur:
            while cur:
                stk.append(cur)
                cur = cur.left
            cur = stk.pop()
            res.append(cur.val)
            cur = cur.right
        return res

## 19.删除链表的倒数第N个节点

dumb节点的重要性

In [70]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [71]:
class Solution:
    def removeNthFromEnd(self, head: Optional[ListNode], n: int) -> Optional[ListNode]:
        p1 = p2 = dumb = ListNode()
        dumb.next = head
        for _ in range(n):
            p2 = p2.next
        while p2.next:
            p1 = p1.next
            p2 = p2.next
        p1.next = p1.next.next
        return dumb.next

## 72. 编辑距离

> 想想二维的问题该如何递推：对于dp[i][j] 的最好情况，一定是从相邻的 dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1] 转移过来，分别考虑转移条件与dp如何更新即可。针对此题只需注意在word1[i] == word2[j] 时的特殊情况，其余状态直接取最小再转移即可。

仔细想想其实这题是一个马尔科夫链模型，每一步的最优情况均从上一步的最优情况转移而来。重点是如何才能想到构建二维dp这个模式。

In [110]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        
        m = len(word1)
        n = len(word2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0: dp[i][j] = j
                if j == 0: dp[i][j] = i
        
        # print(dp)
        
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if word1[i - 1] == word2[j - 1]: dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1] - 1) + 1
                else: dp[i][j] = min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1]) + 1
        
        # print(dp)
        return dp[-1][-1]

In [111]:
word1 = "horse"
word2 = "ros"

In [112]:
s = Solution()
s.minDistance(word1, word2)

3

## 232. 用栈实现队列

In [116]:
class MyQueue:

    def __init__(self):
        self.que = list()
        self.left = 0

    def push(self, x: int) -> None:
        self.que.append(x)

    def pop(self) -> int:
        if not self.empty():
            self.left += 1
            return self.que[self.left - 1]

    def peek(self) -> int:
        if not self.empty():
            return self.que[self.left]

    def empty(self) -> bool:
        return bool(len(self.que) == self.left)

### 双栈

将一个栈当作输入栈，用于压入 push 传入的数据；另一个栈当作输出栈，用于 pop 和 peek 操作。
每次 pop 或 peek 时，若输出栈为空则将输入栈的全部数据依次弹出并压入输出栈，这样输出栈从栈顶往栈底的顺序就是队列从队首往队尾的顺序。

In [117]:
class MyQueue:

    def __init__(self):
        self.stk = list()
        self.que = list()

    def push(self, x: int) -> None:
        self.stk.append(x)

    def pop(self) -> int:
        if not self.que:
            while self.stk:
                self.que.append(self.stk.pop())
        return self.que.pop()
        
    def peek(self) -> int:
        if not self.que:
            while self.stk:
                self.que.append(self.stk.pop())
        return self.que[-1]

    def empty(self) -> bool:
        return not bool(self.stk or self.que)

## 704. 二分查找

In [120]:
class Solution:
    def search(self, nums: List[int], target: int) -> int:
        left = 0
        right = len(nums)
        while left < right:
            mid = (left + right) >> 1
            if nums[mid] == target: return mid
            elif nums[mid] < target: left = mid + 1
            else: right = mid
        return -1

### bisect

In [121]:
def bisect_left(a, x):
    left, right = 0, len(a)
    while left < right:
        mid = (left + right) >> 1
        if a[mid] < x: left = mid + 1
        else: right = mid
    return left

In [122]:
a = [1,2,2,2,2,2,3]; x = 2
bisect_left(a, x)

1

In [123]:
def bisect_right(a, x):
    left, right = 0, len(a)
    while left < right:
        mid = (left + right) >> 1
        if a[mid] <= x: left = mid + 1
        else: right = mid
    return left

In [124]:
a = [1,2,2,2,2,2,3]; x = 2
bisect_right(a, x)

6

## 199. 二叉树的右视图

In [125]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [128]:
from collections import deque
class Solution:
    def rightSideView(self, root: Optional[TreeNode]) -> List[int]:
        if not root: return []
        stk = deque([root]); res = []
        while stk:
            for _ in range(len(stk)):
                cur = stk.popleft()
                if cur.left: stk.append(cur.left)
                if cur.right: stk.append(cur.right)
            res.append(cur.val)
        return res

## 56. 合并区间

In [137]:
class Solution:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        intervals.sort(reverse=True)
        res = [intervals.pop()]
        while intervals:
            l, r = intervals.pop()
            if res[-1][-1] >= l:
                res[-1][-1] = max(res[-1][-1], r)
            else: res.append([l, r])
        return res

In [138]:
intervals = [[1,4],[4,5]]

In [139]:
s = Solution()
s.merge(intervals)

[[1, 5]]

## 70. 爬楼梯

如何转化为dp思维？想想怎么转移就可以，在本题中：每次可以爬1或2个台阶，那么设我们目前在 x 台阶处，则可能会由 x - 2（爬两阶）或x - 1（爬一阶）转移而来，因此到达x的方案数可以拆解为到达x - 2的方案数加上到达x - 1的方案数，即划分为了子问题，以此类推直到边界情况，即：
dp[-2] = 0, dp[-1] = 0, dp[0] = 1。

循环即可计算出 n 的情况

In [142]:
class Solution:
    def climbStairs(self, n: int) -> int:
        p, q, r = 0, 0, 1
        for _ in range(n):
            # 滚动往后推一个状态
            p = q
            q = r
            r = p + q
        return r

## 148. 排序链表

In [143]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [144]:
class Solution:
    def sortList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        cur = dumb = ListNode()
        stk = []
        while head:
            stk.append(head)
            head = head.next
        stk.sort(reverse = True, key = lambda x: x.val)
        while stk:
            cur.next = stk.pop()
            cur = cur.next
        cur.next = None
        return dumb.next

### 自底向上的归并排序

In [146]:
class Solution:
    def sortList(self, head: ListNode) -> ListNode:
        
        # 合并两个有序链表并返回头节点
        def merge(head1: ListNode, head2: ListNode) -> ListNode:
            dummyHead = ListNode(0)
            temp, temp1, temp2 = dummyHead, head1, head2
            while temp1 and temp2:
                if temp1.val <= temp2.val:
                    temp.next = temp1
                    temp1 = temp1.next
                else:
                    temp.next = temp2
                    temp2 = temp2.next
                temp = temp.next
            if temp1:
                temp.next = temp1
            elif temp2:
                temp.next = temp2
            return dummyHead.next
        
        if not head:
            return head
        
        length = 0
        node = head
        while node:
            length += 1
            node = node.next
        
        dummyHead = ListNode(0, head)
        subLength = 1
        while subLength < length:
            prev, curr = dummyHead, dummyHead.next
            while curr:
                
                # 找归并排序的第一个子链表
                head1 = curr
                for i in range(1, subLength):
                    if curr.next:
                        curr = curr.next
                    else:
                        break
                head2 = curr.next
                curr.next = None  # 断尾
                
                # 找归并排序的第二个子链表
                curr = head2
                for i in range(1, subLength):
                    if curr and curr.next:
                        curr = curr.next
                    else:
                        break
                
                # 如果有后继就继合并
                succ = None
                if curr:
                    succ = curr.next
                    curr.next = None
                
                # 合并找到的两链表
                merged = merge(head1, head2)
                
                # 将已排序的链表与未排序的链表连接起来
                prev.next = merged
                while prev.next:
                    prev = prev.next
                curr = succ
            
            # 自底向上归并
            subLength <<= 1
        
        return dummyHead.next

In [150]:
class Solution:
    def sortList(self, head: Optional[ListNode]) -> Optional[ListNode]:
        
        def merge(l1, l2):  # 合并两个有序链表
            cur = dumb = ListNode()
            while l1 and l2:
                if l1.val <= l2.val: cur.next = l1; l1 = l1.next
                else: cur.next = l2; l2 = l2.next
                cur = cur.next
            if l1: cur.next = l1
            if l2: cur.next = l2
            return dumb.next
        
        # 计算链表长度
        length = 0; node = head
        while node:
            length += 1; node = node.next
            
        dumbhead = ListNode(0, head); sub = 1
        while sub < length:
            pre = dumbhead; cur = dumbhead.next
            while cur:
                # 找第一个链表
                head1 = cur
                for _ in range(1, sub):
                    if not cur or not cur.next: break
                    cur = cur.next
                
                head2 = cur.next
                cur.next = None  # 断尾
                
                # 找第二个链表
                cur = head2
                for _ in range(1, sub):
                    if not cur or not cur.next: break
                    cur = cur.next
                
                # 考虑后继链表
                succ = None
                if cur: succ = cur.next; cur.next = None
                
                chead = merge(head1, head2) # 合并两有序链表
                
                # pre指针移动到已经合并完的链表的最后方
                pre.next = chead
                while pre.next:
                    pre = pre.next
                cur = succ  # cur指针指向未排序的第一个节点
            
            sub <<= 1  # sub长度变二倍
        
        return dumbhead.next

## 82. 删除排序链表中的重复元素Ⅱ

In [151]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [155]:
class Solution:
    def deleteDuplicates(self, head: Optional[ListNode]) -> Optional[ListNode]:
        if not head: return 
        dumb = ListNode(); dumb.next = head
        pre = dumb; cur = head; nxt = head.next
        while nxt:
            flag = 0
            while nxt and nxt.val == cur.val:
                flag = 1
                nxt = nxt.next
            if flag: pre.next = nxt
            else: pre = pre.next
            cur = nxt
            if nxt: nxt = nxt.next
        return dumb.next

## 31. 下一个排列

1. 先找出最大的索引 k 满足 nums[k] < nums[k + 1]，如果不存在就翻转整个数组
2. 找出满足 nums[l] > nums[k] 的最大索引 l 
3. 交换 nums[l] 与 nums[k]
4. 由于 l 满足 2 中的条件，故交换后 k 之后的数组必定是倒序排列，为得到最小的变化幅度，将 k 之后的数组翻转得到结果。

如对于 [1,2,7,4,3,1]

1. 找到 k = 1
2. 找到 l = 4
3. 交换得到 nums = [1,3,7,4,2,1]
4. 翻转 1 之后的数组 nums = [1,3,1,2,4,7]

In [206]:
class Solution:
    def nextPermutation(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        left, right = 0, len(nums) - 1
        flag = 0
        for i in range(len(nums) - 1, 0, -1):
            if nums[i] > nums[i - 1]:
                flag = 1
                left = i - 1
                for j in range(len(nums) - 1, i - 1, -1):
                    if nums[j] > nums[i - 1]:
                        right = j
                        break
                break
        if flag: nums[left], nums[right] = nums[right], nums[left]
        print(nums)
        l = left + flag; r = len(nums) - 1
        while l < r:
            nums[l], nums[r] = nums[r], nums[l]
            l += 1; r -= 1
        return nums

In [207]:
nums = [2,3,1]

In [208]:
s = Solution()
s.nextPermutation(nums)

[3, 2, 1]


[3, 1, 2]

## 1143. 最长公共子序列

In [238]:
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        m, n = len(text1), len(text2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if text1[i - 1] == text2[j - 1]: dp[i][j] = dp[i - 1][j - 1] + 1
                # 不相等时不用与 [i - 1][j - 1] 比较，因为这个值肯定较小
                else: dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
                
        # print(dp)
        return dp[-1][-1]

In [239]:
text1 = "bsbininm"
text2 = "jmjkbkjkv"

In [240]:
s = Solution()
s.longestCommonSubsequence(text1, text2)

1

## 69. x的平方根

二分的写法：
- 若判断条件为小于等于，则必须两端同时缩小边界，否则会卡在左右指针重合的情况，同时右边界必须紧贴数组，否则也会越界。
- 使用下除时left指针必须在判断时等于mid + 1，否则left也会一直卡住不动。

In [245]:
class Solution:
    def mySqrt(self, x: int) -> int:
        left, right = 0, x
        while left <= right:
            mid = (left + right) >> 1
            if mid * mid <= x: 
                ans = mid
                left = mid + 1
            else: right = mid
        return ans

### 二分查找的两种写法：

In [247]:
def bisect1(a, x):
    l, r = 0, len(a)  # 宽松右边界
    while l < r:  # 小于
        mid = (l + r) >> 1
        if a[mid] == x: return mid
        elif a[mid] < x: l = mid + 1
        else: r = mid  # 不用 -1 
    return -1

In [248]:
def bisect2(a, x):
    l, r = 0, len(a) - 1
    while l <= r:
        mid = (l + r) >> 1
        if a[mid] == x: return mid
        elif a[mid] < x: l = mid + 1
        else: r = mid - 1
    return -1

## 93. 复原IP地址

In [285]:
class Solution:
    def restoreIpAddresses(self, s: str) -> List[str]:
        def dfs(idx, path):
            # print(path)
            if len(path) > 4: return
            if idx == n:
                if len(path) == 4: ans.append(".".join(path))
                return
            if s[idx] == "0": dfs(idx + 1, path + ["0"])
            else:
                for i in range(idx, n):
                    if int(s[idx: i + 1]) < 256:
                        dfs(i + 1, path + [s[idx: i + 1]])
        
        ans = []; n = len(s)
        dfs(0, [])
        return ans

In [286]:
s = "25525511135"

In [287]:
S = Solution()
S.restoreIpAddresses(s)

['255.255.11.135', '255.255.111.35']

## 41. 缺失的第一个正数

实际上，对于一个长度为 $N$ 的数组，其中没有出现的最小正整数只能在  $[1, N + 1]$ 中。这是因为如果 $[1, N]$ 都出现了，那么答案是 $ N + 1$，否则答案是 $[1, N]$ 中没有出现的最小正整数。

由于我们只关心 $[1, N]$ 之间的数字，因此事先遍历一遍数组，将其中的负数全部改为 $ N + 1 $，从而使得数组中没有负数，这样一来我们就可以使用负号作为标记，来判断对应数组下标的那个数字有没有出现过。

In [12]:
from typing import List
class Solution:
    def firstMissingPositive(self, nums: List[int]) -> int:
        n = len(nums)
        for i in range(n):
            if nums[i] <= 0: nums[i] = n + 1
        
        for i in range(n):
            num = abs(nums[i])
            if num <= n:
                nums[num - 1] = -abs(nums[num - 1])
        
        for i in range(n):
            if nums[i] > 0: return i + 1
        
        return n + 1

In [13]:
nums = [3,4,-1,1]

In [14]:
s = Solution()
s.firstMissingPositive(nums)

2

## 239. 滑动窗口最大值

pass

## 剑指 Offer 22. 链表中倒数第k个节点

In [15]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

In [16]:
class Solution:
    def getKthFromEnd(self, head: ListNode, k: int) -> ListNode:
        dumb = ListNode(); dumb.next = head; tail = dumb
        for _ in range(k): 
            tail = tail.next
        while tail:
            dumb = dumb.next
            tail = tail.next
        return dumb

## 165. 比较版本号

In [18]:
class Solution:
    def compareVersion(self, version1: str, version2: str) -> int:
        v1 = version1.split("."); v2 = version2.split(".")
        m = len(v1); n = len(v2)
        if m > n: v2.extend(["0"] * (m - n))
        if m < n: v1.extend(["0"] * (n - m))
        
        for a, b in zip(*(v1, v2)):
            if int(a) > int(b): return 1
            if int(a) < int(b): return -1
        
        return 0

## 105. 从前序与中序遍历序列构造二叉树

In [19]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [21]:
from typing import Optional

class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        def build(preleft, preright, inleft, inright):
            # 目前树的节点对应 preorder 与 inorder 的下标
            if preleft > preright: return  # 树已经建完
            RootVal = preorder[preleft]  # 根节点的取值，即为 pre 遍历的第一个值
            curRoot = TreeNode(RootVal)  # 建立当前的根节点
            RootIdx = query[RootVal]  # 在 inorder 中定位到当前的根节点，左侧即为左子树，右侧为右子树
            curRoot.left = build(
                preleft + 1,
                preleft + RootIdx - inleft,  # 前序遍历中根节点后面跟的是全部的左子树，节点数目可由中序遍历的 RootIdx 确定
                inleft, 
                RootIdx - 1
            )
            curRoot.right = build(
                preleft + RootIdx - inleft + 1,
                preright,  # 左子树后面跟的就是右子树
                RootIdx + 1,
                inright
            )
            return curRoot  # 返回建立的根节点
        
        n = len(inorder)
        query = {num: i for i, num in enumerate(inorder)}
        return build(0, n - 1, 0, n - 1)

## 151. 翻转字符串中的单词

In [35]:
class Solution:
    def reverseWords(self, s: str) -> str:
        s = s.strip()
        s = s.split(" ")
        res = []
        while s:
            cur = s.pop().strip()
            if cur: res.append(cur)
        return " ".join(res)

In [36]:
s = "a good   example"

In [37]:
S = Solution()
S.reverseWords(s)

'example good a'

## 322. 零钱兑换

In [38]:
from functools import lru_cache
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        @lru_cache(2000)
        def dfs(rem):
            if rem < 0: return -1  # 不能完成兑换
            if rem == 0: return 0  # 刚好完成兑换
            minstep = float("inf")
            for c in coins:
                cur = dfs(rem - c)
                if cur >= 0: minstep = min(minstep, cur + 1)
            return minstep if minstep != float("inf") else -1
                
        return dfs(amount)

In [50]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        dp = [float("inf")] * (amount + 1); dp[0] = 0
        for i in range(amount + 1):
            for c in coins:
                if i - c >= 0: dp[i] = min(dp[i], dp[i - c] + 1)
        return dp[-1] if dp[-1] != float("inf") else -1

In [51]:
coins = [186,419,83,408]
amount = 6249

In [52]:
s = Solution()
s.coinChange(coins, amount)

20

## 113. 路径总和Ⅱ

In [4]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

每次访问子节点都生成一个新的path对象，没有用到回溯的思想

In [5]:
from typing import Optional, List
class Solution:
    def pathSum(self, root: Optional[TreeNode], targetSum: int) -> List[List[int]]:
        def dfs(node: Optional[TreeNode], path: List[int]):
            if not node: return
            if not node.left and not node.right:
                if sum(path) + node.val == targetSum: res.append(path + [node.val])
                return
            dfs(node.left, path + [node.val])
            dfs(node.right, path + [node.val])
        
        res = []
        dfs(root, [])
        return res 

利用了回溯的思想，每次访问完某节点回溯时在path中pop该结点的值，实现一个path列表的复用，减小空间开销

In [6]:
class Solution:
    def pathSum(self, root: Optional[TreeNode], targetSum: int) -> List[List[int]]:
        def dfs(node, curSum):
            if not node: return
            path.append(node.val)
            curSum += node.val
            if not node.left and not node.right:
                if curSum == targetSum: res.append(path[:])
            dfs(node.left, curSum)
            dfs(node.right, curSum)
            path.pop()
        
        res = []; path = []
        dfs(root, 0)
        return res 

## 98. 验证二叉搜索树

In [3]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

1. 二叉搜索树的中序遍历为单调不减的序列

In [11]:
class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        def dfs(node):
            nonlocal judge
            if not node: return
            dfs(node.left)
            if res and res[-1] >= node.val: judge = False 
            res.append(node.val)
            dfs(node.right)

        res = []; judge = True 
        dfs(root)
        return judge 

2. 递归

1. 节点的左子树只包含 小于 当前节点的数。
2. 节点的右子树只包含 大于 当前节点的数。
3. 所有左子树和右子树自身必须也是二叉搜索树。

In [11]:
from typing import Optional
class Solution:
    def isValidBST(self, root: Optional[TreeNode]) -> bool:
        def judge(min_, max_, node):
            if not node: return True  # 节点为空的情况
            cur = node.val
            if not (min_ < cur < max_): return False  # 条件 1，2
            flag1 = judge(min_, cur, node.left)
            flag2 = judge(cur, max_, node.right)
            return flag1 and flag2  # 条件 3

        return judge(float("-inf"), float("inf"), root)

## 64. 最小路径和

In [49]:
from typing import List

class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        row, col = len(grid), len(grid[0])
        dp = [[grid[0][0] for _ in range(col)] for _ in range(row)]
        get_dp = lambda i, j : float("inf") if (i < 0 or j < 0) else dp[i][j]
        for i in range(row):
            for j in range(col):
                if i == j == 0: continue
                dp[i][j] = min(get_dp(i - 1, j), get_dp(i, j - 1)) + grid[i][j]
        return dp[-1][-1]

In [50]:
grid = [[1,3,1],[1,5,1],[4,2,1]]

In [51]:
s = Solution()
s.minPathSum(grid)

7

## 48. 旋转图像

In [2]:
from typing import List 
class Solution:
    def rotate(self, matrix: List[List[int]]) -> None:
        row, col = len(matrix), len(matrix[0])
        # 对角线翻转
        for i in range(row):
            for j in range(i, col):
                if i == j: continue
                matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]
        # 水平翻转
        for i in range(col // 2):
            for j in range(row):
                matrix[j][i], matrix[j][col - i - 1] = \
                    matrix[j][col - i - 1], matrix[j][i]
        

## 39. 组合总和

In [9]:
from collections import defaultdict
class Solution:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        def dfs(idx, curSum, path):
            if curSum >= target:
                if curSum == target: res.append(path)
                return 
            # 注意这里从 i 开始，意味着可以无限选择同一个数字
            for i in range(idx, n):
                dfs(i, curSum + candidates[i], path + [candidates[i]])
        
        res = []
        n = len(candidates)
        dfs(0, 0, [])
        return res 

## 394. 字符串解码

数字存放在数字栈，字符串存放在字符串栈，遇到右括号时候弹出一个数字栈，字母栈弹到左括号为止。就是逆波兰式那种题。

In [130]:
class Solution:
    def decodeString(self, s: str) -> str:
        stk_n = []
        stk_c = []
        n_buf = "" 
        for ch in s:
            if ch == "]":
                cur = ""
                p = stk_c.pop()
                while p != "[":
                    cur += p[::-1]  # 注意这里一定要反着弹出来
                    p = stk_c.pop()
                cur = cur[::-1]  # 这里也要取反
                cur *= stk_n.pop()
                stk_c.append(cur)
            elif ch.isdigit(): n_buf += ch 
            else:
                if ch == "[":
                    stk_n.append(int(n_buf))
                    n_buf = ""
                stk_c.append(ch)
            # print(stk_c)

        return "".join(stk_c)

## 221. 最大正方形

In [161]:
class Solution:
    def maximalSquare(self, matrix: List[List[str]]) -> int:
        m, n = len(matrix), len(matrix[0])
        res = 0
        dp = [[0] * n for _ in range(m)]
        get_dp = lambda r, c : 0 if (r < 0 or c < 0) else dp[r][c]
        for i in range(m):
            for j in range(n):
                if matrix[i][j] == "1":
                    dp[i][j] = min(get_dp(i, j - 1), get_dp(i - 1, j), 
                                   get_dp(i - 1, j - 1)) + 1
                    res = max(res, dp[i][j])
        return res ** 2

## 34. 在排序数组中查找元素的第一个和最后一个位置

In [162]:
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        
        def bisect_left(a, x):
            l, r = 0, len(a)
            while l < r:
                mid = (l + r) // 2
                # 由于是左查找，因此最后一定要停在第一个大于等于 x 的数字的 idx
                # 因此要先判断不符合的情况，如果 mid 小于 x 的话直接从 mid + 1 开始向右搜
                # 但是针对 mid 大于等于 x 的情况则不能舍弃这个 mid，因此要以 mid 为右边界而不是 mid - 1
                # 另。由于 // 取得是下界，因此 l 必须要为 mid + 1，否则会陷入循环，因此无论是左搜还是右搜都要设计使得
                # l = mid + 1 的条件
                if a[mid] < x: l = mid + 1
                else: r = mid
            return l 
        
        def bisect_right(a, x):
            l, r = 0, len(a)
            while l < r:
                # 二者的区别仅在于判断时是 < 还是 <= 
                # 显然地，想要找到最右边界要使用 <= 作为判断条件
                mid = (l + r) // 2
                if a[mid] <= x: l = mid + 1
                else: r = mid
            return l 
        
        a, b = bisect_left(nums, target), bisect_right(nums, target)
        # 特判是精髓，bisectleft返回的idx一定是原数组中最接近 x 的那个数字的 idx，除非当前数组的所有数字均小于 x
        # 此时会插在最后一个位置，会越界，因此要用 min 来特判
        if not nums or nums[min(a, len(nums) - 1)] != target: return [-1, -1]
        return [a, b - 1]
        

## 718. 最长重复子数组

dp

In [164]:
class Solution:
    def findLength(self, nums1: List[int], nums2: List[int]) -> int:
        m, n = len(nums1), len(nums2)
        dp = [[0] * n for _ in range(m)]
        res = 0
        get_dp = lambda r, c : 0 if (r < 0 or c < 0) else dp[r][c]
        for i in range(m):
            for j in range(n):
                if nums1[i] == nums2[j]: 
                    dp[i][j] = get_dp(i - 1, j - 1) + 1
                    res = max(res, dp[i][j])
        return res 

- 时间复杂度: $O(M \times N)$
- 空间复杂度: $O(M \times N)$

滑窗

In [165]:
class Solution:
    def findLength(self, A: List[int], B: List[int]) -> int:
        def maxLength(addA: int, addB: int, length: int) -> int:
            # addA, addB 代表两数组各自的偏移量
            # length 代表重叠区间长度
            # 返回此种重叠方式下的最长重复子数组长度
            ret = k = 0
            for i in range(length):
                if A[addA + i] == B[addB + i]:
                    k += 1
                    ret = max(ret, k)
                else:
                    k = 0
            return ret
        
        n, m = len(A), len(B)
        ret = 0
        for i in range(n):
            length = min(m, n - i)  # 重叠的区间长度
            ret = max(ret, maxLength(i, 0, length))
        for i in range(m):
            length = min(n, m - i)
            ret = max(ret, maxLength(0, i, length))
        return ret

In [ ]:
class Solution:
    def findLength(self, nums1: List[int], nums2: List[int]) -> int:
        def getRepeatLength(addA, addB, length):
            res = cur = 0
            for i in range(length):
                if nums1[addA + i] == nums2[addB + i]:
                    cur += 1
                    res = max(res, cur)
                else: cur = 0
            return res 
        
        m, n = len(nums1), len(nums2)
        res = 0

        for i in range(m):
            res = max(res, getRepeatLength(i, 0, min(n, m - i)))
        for i in range(n):
            res = max(res, getRepeatLength(0, i, min(m, n - i)))

        return res

- 时间复杂度: $O((N + M) \times \min{(N, M)})$
- 空间复杂度: $O(1)$

## 240. 搜索二维矩阵

In [180]:
class Solution:
    def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
        m, n = len(matrix), len(matrix[0])
        x, y = 0, n - 1
        # 搜索条件为在矩阵范围之内
        while x <= m - 1 and y >= 0:
            if matrix[x][y] == target: return True
            if matrix[x][y] < target: x += 1
            else: y -= 1
        return False

## 162. 寻找峰值

总之要一直向高处走，若 $nums[i + 1] > nums[i]$, 那么向右走一定能找到峰值。若寻找谷底则是一直向低处走

In [1]:
from typing import List 
class Solution:
    def findPeakElement(self, nums: List[int]) -> int:
        get = lambda idx : float("-inf") if (idx == -1 or idx == n) else nums[idx]
        def find(l, r):
            if l > r: return l
            mid = (l + r) // 2
            return find(l, mid - 1) if get(mid) > get(mid + 1) else find(mid + 1, r)
        n = len(nums)
        return find(0, n - 1)


## 227. 基本计算器Ⅱ

重点是维护前一个字符，这样才能做到优先运算乘除

In [21]:
class Solution:
    def calculate(self, s: str) -> int:
        n = len(s)
        stack = []
        preSign = '+'  # 维护上一个字符，妙！
        num = 0
        for i in range(n):
            if s[i] != ' ' and s[i].isdigit():
                num = num * 10 + ord(s[i]) - ord('0')
            if i == n - 1 or s[i] in '+-*/':
                if preSign == '+':
                    stack.append(num)
                elif preSign == '-':
                    stack.append(-num)
                elif preSign == '*':
                    stack.append(stack.pop() * num)
                else:
                    stack.append(int(stack.pop() / num))
                preSign = s[i]
                num = 0
        return sum(stack)

In [42]:
class Solution:
    def calculate(self, s: str) -> int:
        n = len(s)
        stack = []
        preSign = '+'
        num = 0
        for i, ch in enumerate(s):
            # print(preSign, stack)
            if ch != ' ' and ch.isdigit(): num = num * 10 + int(ch)
            if i == n - 1 or ch in '+-*/':
                if preSign == '+': stack.append(num)
                elif preSign == '-': stack.append(-num)
                elif preSign == '*': stack.append(stack.pop() * num)
                # // 会向更小的方向取整，对于负数除法会出错，使用 / 再 int 能避免这个问题
                else: stack.append(int(stack.pop() / num))
                preSign = ch 
                num = 0
        return sum(stack)

## 128. 最长连续序列

In [46]:
from typing import List
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        nums, res = set(nums), 0
        for n in nums:
            # 方案为一直向大的数字枚举，因此如果一个数的前驱在set中，那么就不用重复枚举
            if n - 1 in nums: continue
            cur = n
            while cur + 1 in nums: cur += 1
            res = max(res, cur - n + 1)
        return res

## 695. 岛屿的最大面积

In [2]:
from typing import List 
class Solution:
    def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
        def dfs(r, c):
            ds = ((0, -1), (0, 1), (-1, 0), (1, 0))
            res = 1
            grid[r][c] = 0
            for dr, dc in ds:
                nr, nc = r + dr, c + dc
                if 0 <= nr < m and 0 <= nc < n:
                    if grid[nr][nc] == 1:
                        res += dfs(nr, nc)
            return res 
        
        m, n = len(grid), len(grid[0])
        ans = 0
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 1: ans = max(dfs(i, j), ans)
        
        return ans 

In [6]:
grid = [[0,0,1,0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,1,1,0,1,0,0,0,0,0,0,0,0],[0,1,0,0,1,1,0,0,1,0,1,0,0],[0,1,0,0,1,1,0,0,1,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,1,1,0,0,0,0]]
grid = [[0,0,0,0,0,0,0,0]]

In [7]:
s = Solution()
s.maxAreaOfIsland(grid)

0

## 198. 打家劫舍

In [14]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        dp = [[0, 0], [0, 0]]
        ans = 0
        for n in nums:
            cur = [0, 0]
            cur[0] = max(dp[0][1], dp[1][1])
            cur[1] = max(dp[0][1], dp[1][0]) + n 
            ans = max(ans, cur[0], cur[1])
            dp[0], dp[1] = dp[1], cur 
        return ans 

In [21]:
nums = [10,2,3,10]

In [22]:
s = Solution()
s.rob(nums)

20

In [23]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        dp = [0, 0]; ans = 0
        for n in nums:
            cur = max(dp[1], dp[0] + n)
            ans = max(ans, cur)
            dp[0], dp[1] = dp[1], cur 
        return ans 

## 139. 单词拆分

In [ ]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        def dfs(idx):
            if idx > n - 1: return True
            if idx in memo: return memo[idx]
            ans = False
            for i in range(idx + 1, n + 1):
                if s[idx:i] in wordDict: ans |= dfs(i)
            memo[idx] = ans 
            return ans 
        
        n = len(s)
        wordDict = set(wordDict)
        memo = {}
        return dfs(0)

## 662. 二叉树最大宽度

In [1]:
# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

In [3]:
from typing import Optional
from collections import deque
class Solution:
    def widthOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        que = deque([(root, 0)])
        ans = 0
        while que:
            idxs = []
            for _ in range(len(que)):
                cur, idx = que.popleft()
                idxs.append(idx)
                if cur.left: que.append((cur.left, 2 * idx))
                if cur.right: que.append((cur.right, 2 * idx + 1))
                ans = max(ans, idxs[-1] - idxs[0])
        return ans + 1

## 152. 乘积最大子数组

In [39]:
from typing import List 
class Solution:
    def maxProduct(self, nums: List[int]) -> int:
        pre0 = pre1 = 1
        ans = nums[0]
        for n in nums:
            cur0 = max(n, pre0 * n, pre1 * n)
            cur1 = min(n, pre0 * n, pre1 * n)
            pre0, pre1 = cur0, cur1
            ans = max(ans, pre0) 
        return ans 

In [42]:
nums = [-2]

In [43]:
s = Solution()
s.maxProduct(nums)

-2

## [179. 最大数](https://leetcode.cn/problems/largest-number/)

In [44]:
from functools import cmp_to_key
from typing import List

class Solution:
    def largestNumber(self, nums: List[int]) -> str:
        def comp(num1, num2):
            return int(num2 + num1) - int(num1 + num2)
    
        nums = [str(n) for n in nums]
        res = "".join(sorted(nums, key=cmp_to_key(comp)))
        return "0" if res[0] == "0" else res

In [41]:
nums = [3,30,34,5,9]

In [42]:
s = Solution()
s.largestNumber(nums)

'9534330'

## [24. 两两交换链表中的节点](https://leetcode.cn/problems/swap-nodes-in-pairs/)

In [48]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

from typing import Optional

class Solution:
    def swapPairs(self, head: Optional[ListNode]) -> Optional[ListNode]:
        if not head or not head.next: return head
        dumb = ListNode(head)
        pre, fir, sec, end = dumb, head, head.next, None
        while sec:
            end = sec.next
            pre.next = sec
            sec.next = fir
            fir.next = end
            pre = fir
            fir = end
            if end: sec = end.next
            else: break

        return dumb.next

## [560. 和为 K 的子数组](https://leetcode.cn/problems/subarray-sum-equals-k/)

In [74]:
class Solution:
    def subarraySum(self, nums: List[int], k: int) -> int:
        preSum = {0: 1}
        curSum = 0
        ans = 0
        for n in nums:
            curSum += n
            if curSum - k in preSum: ans += preSum[curSum - k]
            preSum[curSum] = preSum.get(curSum, 0) + 1
        return ans

In [75]:
nums = [1,2,1,2,1]
k = 3

In [76]:
s = Solution()
s.subarraySum(nums, k)

4

## [233. 数字 1 的个数](https://leetcode.cn/problems/number-of-digit-one/description/)

熟悉的数位dp！

In [4]:
class Solution:
    def countDigitOne(self, n: int) -> int:
        digits = [int(i) for i in str(n)]
        dp = dict()

        def dfs(pos : int, isBound: bool, cnt : int) -> int:
            if (pos == len(digits)): return cnt
            if (pos, isBound, cnt) in dp: return dp[(pos, isBound, cnt)]
            ans = 0
            for i in range(digits[pos] + 1 if isBound else 10):
                ans += dfs(
                    pos + 1,
                    isBound and i == digits[pos],
                    cnt + 1 if i == 1 else cnt
                )
            dp[(pos, isBound, cnt)] = ans
            return ans

        return dfs(0, True, 0)

## [572. 另一棵树的子树](https://leetcode.cn/problems/subtree-of-another-tree/description/)

In [14]:
from typing import Optional

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


class Solution:
    def isSubtree(self, root: Optional[TreeNode], subRoot: Optional[TreeNode]) -> bool:
        def isSame(root1, root2):
            if ((root1 == None) ^ (root2 == None)): return False
            elif (root1 == None): return True
            else: return (root1.val == root2.val) and \
                isSame(root1.left, root2.left) and isSame(root1.right, root2.right)

        def dfs(node):
            nonlocal res
            if not node: return
            if node.val == subRoot.val:
                res |= isSame(node, subRoot)
            dfs(node.left)
            dfs(node.right)

        res = False
        dfs(root)
        return res

## [42. 接雨水](https://leetcode.cn/problems/trapping-rain-water/description/)

1. 按行计算，每次统计一个高度的雨水共有多少。

    具体方法为：对于一个给定的高度 $h$，遍历所有列表中的元素，如果有高于 $h$ 的元素，则开始计数，表示有左边的墙了。如果元素低于 $h$ 且有左边的墙，则说明这里的水可能可以被接到，加到 temp 中；如果有左边的墙且再次遇到大于 $h$ 的元素，则将 temp 加入到 ans 中，表示这一块已经接到了雨水。

In [21]:
from typing import List

class Solution:
    def trap(self, height: List[int]) -> int:
        maxH = max(height)
        ans = 0
        for i in range(1, maxH + 1):
            temp, bHasWall = 0, False
            for h in height:
                if h >= i:
                    if bHasWall: 
                        ans += temp
                        temp = 0
                    else: bHasWall = True
                else:
                    if bHasWall: temp += 1

        return ans

2. 一个位置能够接收到的最高雨水高度等于左侧最大高度与右侧最大高度的最小值，能接到的雨水量等于该处雨水能够达到的最大高度减去 `height[i]`，预处理出左侧最大高度与右侧最大高度的数组，依次查询即可。

In [3]:
from typing import List

class Solution:
    def trap(self, height: List[int]) -> int:
        maxLeft, maxRight = height[0], height[-1]
        left, right = [0 for _ in range(len(height))], [0 for _ in range(len(height))]
        for i in range(len(height)):
            if height[i] > maxLeft:
                maxLeft = left[i] = height[i]
            else: left[i] = maxLeft

        for i in range(len(height) - 1, -1, -1):
            if height[i] > maxRight:
                maxRight = right[i] = height[i]
            else: right[i] = maxRight
        
        ans = 0
        for i, h in enumerate(height):
            ans += min(left[i], right[i]) - h

        return ans

3. 双指针法：始终移动较小的指针并且记录左侧与右侧的最大值，这样可以保证当 `height[left] < height[right]` 时，必然有 `maxLeft < maxRight`，对称的情况同理，遍历并累计接水量，直到指针相遇。

In [6]:
class Solution:
    def trap(self, height: List[int]) -> int:
        ans = 0
        left, right = 0, len(height) - 1
        leftMax, rightMax = 0, 0

        while left < right:
            leftMax = max(leftMax, height[left])
            rightMax = max(rightMax, height[right])
            if height[left] < height[right]:
                ans += min(leftMax, rightMax) - height[left]
                left += 1
            else:
                ans += min(leftMax, right) - height[right]
                right -= 1
        
        return ans

## [287. 寻找重复数](https://leetcode.cn/problems/find-the-duplicate-number/description/)

1. 二分：**寻找单调性**，题中的单调性如何体现？设重复的数字为 `target`，那么对于小于 `target` 的任意数字 `n`，数组中小于 `n` 的数字数量一定小于 `n`，而大于等于 `target` 的任意数字 `N`，数组中大于 `N` 的数字数量一定大于等于 `N`，具有单调性，二分查找 `traget`，判断的时间复杂度为 `O(n)`，总体时间复杂度 `O(nlogn)`。

In [16]:
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        left, right = 1, len(nums) - 1
        ans = -1
        while left <= right:
            mid = (left + right) // 2
            cnt = 0
            for n in nums:
                cnt += n <= mid
            if cnt <= mid: left = mid + 1
            else:
                right = mid - 1
                ans = mid
        return ans

2. 快慢指针：由于 nums 长度为 `n + 1`，即索引为 `[0, n]`，而其中只包含 `[1, n + 1]` 的数字，数组中的每个元素可以看作是指向下一个元素的指针，其中元素的值代表下一个元素的索引。由于数组中的数字都在[1, n]范围内，且数组长度为n+1，根据鸽巢原理，至少有一个数字是重复的。这种重复意味着至少有两个位置指向了同一个索引，从而形成了一个环。

![image.png](https://s2.loli.net/2024/03/12/n3C4dmozc5Xahw6.png)

找环入口：快慢指针相遇后再使用一个指针从头部开始移动，慢指针同时移动，最终两个指针会在环的入口相遇。

In [2]:
from typing import List

class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        slow = fast = nums[0]
        while True:
            slow = nums[slow]
            fast = nums[nums[fast]]
            if slow == fast: break

        slow = nums[0]  # 从入口开始重新移动
        while slow != fast:
            slow = nums[slow]
            fast = nums[fast]

        return slow

## [16. 最接近的三数之和](https://leetcode.cn/problems/3sum-closest/description/)

In [19]:
class Solution:
    def threeSumClosest(self, nums: List[int], target: int) -> int:
        nums.sort()
        ans = float("Inf")

        def update(cur):
            nonlocal ans
            if abs(cur - target) < abs(ans - target):
                ans = cur

        for i, n in enumerate(nums):
            if (i > 0 and nums[i] == nums[i - 1]): continue
            left, right = i + 1, len(nums) - 1
            while left < right:
                curSum = n + nums[left] + nums[right]
                if curSum == target: return target
                update(curSum)
                if (curSum > target):
                    rightVal = nums[right]
                    while (left < right and nums[right] == rightVal):
                        right -= 1
                else:
                    leftVal = nums[left]
                    while (left < right and nums[left] == leftVal):
                        left += 1
        return ans

In [20]:
nums = [-1,2,1,-4]
target = 1

In [21]:
s = Solution()
s.threeSumClosest(nums, target)

2

## [1011. 在 D 天内送达包裹的能力](https://leetcode.cn/problems/capacity-to-ship-packages-within-d-days/description/)

In [5]:
import heapq
from typing import List

class Solution:
    def shipWithinDays(self, weights: List[int], days: int) -> int:
        weights.sort(reverse=True)
        heap = []
        for w in weights:
            if len(heap) < days: heapq.heappush(heap, w)
            else: heapq.heappush(heap, heapq.heappop(heap) + w)
        return max(heap)

这应该是处理可以乱序情况下的解法？

最后还是要二分做，略微总结一下：面对 “最xxx” 的问题时，先思考如何判断一个特定值是否满足要求，复杂度是多少。“最xxx” 的问题天然具备单调性，即小于某个值的答案均不成立，大于这个值的答案均成立，因此可以使用二分来查找这个临界值。

In [33]:
class Solution:
    def shipWithinDays(self, weights: List[int], days: int) -> int:
        def check(w):
            cnt = 0
            temp = 0
            for n in weights:
                if n >= w:
                    temp = 0
                    cnt += 2
                else:
                    temp += n
                    if temp > w:
                        temp = n
                        cnt += 1
            if temp: cnt += 1
            # print(w, cnt)
            return cnt <= days
        
        left, right = max(weights), sum(weights)
        while left < right:
            mid = (left + right) >> 1
            if check(mid): right = mid
            else: left = mid + 1

        return left

In [34]:
weights = [1,2,3,4,5,6,7,8,9,10]
days = 5

In [35]:
s = Solution()
s.shipWithinDays(weights, days)

15

## [153. 寻找旋转排序数组中的最小值](https://leetcode.cn/problems/find-minimum-in-rotated-sorted-array/description/)

In [3]:
from typing import List

class Solution:
    def findMin(self, nums: List[int]) -> int:
        left, right = 0, len(nums) - 1
        while left < right:
            mid = (left + right) >> 1
            if mid > 0 and nums[mid - 1] > nums[mid] < nums[mid + 1]:
                return nums[mid]
            if nums[mid] <= nums[right]: right = mid
            else: left = mid + 1
        return min(nums[left], nums[0])

## [143. 重排链表](https://leetcode.cn/problems/reorder-list/description/)

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [3]:
from typing import Optional

class Solution:
    def reorderList(self, head: Optional[ListNode]) -> None:
        """
        Do not return anything, modify head in-place instead.
        """
        def middleNode(head):
            slow = fast = head
            while fast.next and fast.next.next:
                slow = slow.next
                fast = fast.next.next
            return slow
        
        def reverseList(head):
            pre, cur = None, head
            while cur:
                nxt = cur.next
                pre = cur.next
                pre = cur
                cur = nxt
            return pre
        
        def mergeList(l1, l2):
            while l1 and l2:
                nxt1, nxt2 = l1.next, l2.next
                l1.next = l2
                l1 = nxt1
                l2.next = l1
                l2 = nxt2

        if not head: return
        mid = middleNode(head)
        l1 = head
        l2 = mid.next
        mid.next = None
        l2 = reverseList(l2)
        mergeList(l1, l2)

## [82. 删除排序链表中的重复元素Ⅱ](https://leetcode.cn/problems/remove-duplicates-from-sorted-list-ii/description/)

In [4]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

In [6]:
class Solution:
    def deleteDuplicates(self, head: Optional[ListNode]) -> Optional[ListNode]:
        dumb = ListNode(0, head)
        cur = dumb
        while cur.next and cur.next.next:
            val = cur.next.val
            if cur.next.next.val == val:
                while cur.next and cur.next.val == val:
                    cur.next = cur.next.next
            else: cur = cur.next
        return dumb.next

In [11]:
class Solution:
    def deleteDuplicates(self, head: Optional[ListNode]) -> Optional[ListNode]:
        if not head: return
        dumb = ListNode(0, head)
        pre, cur = dumb, head
        while cur:
            flag = 0
            # cur.val == next.val 时持续移动 nxt
            nxt = cur.next
            while nxt and cur.val == nxt.val:
                flag = 1
                nxt = nxt.next
            # 有重复情况，直接跳过中间的数字，将 pre.next 和 cur 都赋以 nxt
            if flag == 1: pre.next = cur = nxt
            # 没有重复的情况，pre 和 cur 都向后移动
            if flag == 0: pre, cur = pre.next, cur.next
        return dumb.next

## [438. 找到字符串中所有字母异位词](https://leetcode.cn/problems/find-all-anagrams-in-a-string/description/)

In [6]:
from typing import List

class Solution:
    def findAnagrams(self, s: str, p: str) -> List[int]:
        n1, n2 = len(s), len(p)
        res = []

        if n1 < n2: return res
        
        target = [0] * 26
        cur    = [0] * 26
        
        for c in p:
            target[ord(c) - 97] += 1
        
        for c in s[:n2]:
            cur[ord(c) - 97] += 1
        
        if cur == target: res.append(0)

        for i in range(n1 - n2):
            cur[ord(s[i]) - 97] -= 1
            cur[ord(s[n2 + i]) - 97] += 1
            if cur == target: res.append(i + 1)
        
        return res

## [96. 不同的二叉搜索树](https://leetcode.cn/problems/unique-binary-search-trees/description/)

In [13]:
class Solution:
    def numTrees(self, n: int) -> int:
        dp = [0] * 22
        dp[0] = dp[1] = 1
        for i in range(2, n + 1):
            for j in range(i):
                dp[i] += dp[j] * dp[i - j - 1]
        return dp[n]

In [14]:
s = Solution()
s.numTrees(3)

5

## [139. 单词拆分](https://leetcode.cn/problems/word-break/description/)

In [36]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        def dfs(idx):
            if idx > n - 1: return True
            if idx in memo: return memo[idx]
            ans = False
            for i in range(idx + 1, n + 1):
                if s[idx:i] in wordDict: ans |= dfs(i)
            memo[idx] = ans 
            return ans 
        
        n = len(s)
        wordDict = set(wordDict)
        memo = {}
        return dfs(0)

In [ ]:
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:

        def dfs(idx):
            if idx >=n : return True
            if idx in memo: return memo[idx]
            ans = False
            for i in range(idx + 1, n + 1):
                if s[idx:i] in wordDict: ans |= dfs(i)
            memo[idx] = ans
            return ans

        n = len(s)
        wordDict = set(wordDict)
        memo = dict()
        return dfs(0)

## [131. 分割回文串](https://leetcode.cn/problems/palindrome-partitioning/description/)

In [24]:
from typing import List

class Solution:
    def partition(self, s: str) -> List[List[str]]:
        path = [0]
        res = []
        n = len(s)
        memo = dict()

        def getAns():
            cur = []
            for i in range(0, len(path) - 1):
                cur.append(s[path[i] : path[i + 1]])
            return cur

        def dfs(idx):
            if idx == n:
                if s[path[-1] : idx] == s[path[-1] : idx][::-1]:
                    res.append(getAns())
                return
            
            for i in range(idx + 1, n + 1):
                if s[idx : i] == s[idx : i][::-1]:
                    path.append(i)
                    dfs(i)
                    path.pop()
            
        dfs(0)
        return res

In [38]:
s = "aab"
S = Solution()
S.partition(s)

[['a', 'a', 'b'], ['aa', 'b']]

In [37]:
from typing import List

class Solution:
    def partition(self, s: str) -> List[List[str]]:
        path = [0]
        res = []
        n = len(s)
        dp = [[False for _ in range(n + 1)] for _ in range(n + 1)]

        for i in range(n):
            for j in range(i + 1, n + 1):
                if s[i : j] == s[i : j][::-1]:
                    dp[i][j] = True

        def getAns():
            cur = []
            for i in range(0, len(path) - 1):
                cur.append(s[path[i] : path[i + 1]])
            return cur

        def dfs(idx):
            if idx == n:
                if dp[path[-2]][idx]: 
                    res.append(getAns())
                return
            
            for i in range(idx + 1, n + 1):
                if dp[idx][i]:
                    path.append(i)
                    dfs(i)
                    path.pop()
            
        dfs(0)
        return res

In [ ]:
class Solution:
    def partition(self, s: str) -> List[List[str]]:
        n = len(s)
        f = [[True] * n for _ in range(n)]

        # 提前计算好是否为回文串
        for i in range(n - 1, -1, -1):
            for j in range(i + 1, n):
                f[i][j] = (s[i] == s[j]) and f[i + 1][j - 1]

        ret = list()
        ans = list()

        def dfs(i: int):
            if i == n:
                ret.append(ans[:])
                return
            
            for j in range(i, n):
                if f[i][j]:
                    ans.append(s[i:j+1])
                    dfs(j + 1)
                    ans.pop()

        dfs(0)
        return ret